In [1]:
import sys
import time
sys.path.insert(0, '/home/anish/ASC_ML')

In [2]:
from ASC_ML.networkbuilding import multiple_model_gen_v3 as multiple
from ASC_ML.networkbuilding import dataframe_extractor as de
from ASC_ML.networkbuilding import model_generation as model_gen
# from ASC_ML import model_gen_train_test as testing
from dask_ml.preprocessing import LabelEncoder
# from dask_ml.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

2022-08-18 16:19:40.248927: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
# m = multiple.Multiple_Model_Gen(max_no_layers = 2, model_per_batch = 10)

In [4]:
# import dask.dataframe as dd
# import pandas as pd
# data = dd.from_pandas(pd.Series(['a', 'a', 'b'], dtype='category'),
#                      npartitions=1)
# le = LabelEncoder()
# le.fit_transform(data).compute()
# print(type(le.fit_transform(data).compute()))

In [5]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"


# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1, label_names = ["SalePrice"]).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

Reading single csv from /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv
Train Dataset X Columns


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal


In [6]:
dataset_list[1].head()

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0


In [7]:
train_x = dataset_list[0].copy()

In [8]:
train_x = train_x.fillna(0)

In [9]:
train_x.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [10]:
train_x['MSSubClass'] = train_x['MSSubClass'].apply(str)

train_x['OverallCond'] = train_x['OverallCond'].astype(str)

train_x['YrSold'] = train_x['YrSold'].astype(str)
train_x['MoSold'] = train_x['MoSold'].astype(str)
train_x.head()

/home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/dask/dataframe/core.py:3946: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('MSSubClass', 'object'))

  warnings.warn(meta_warning(meta))


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [11]:
# a = train_x['LotShape']

In [12]:
# le = LabelEncoder() 
# le.fit_transform(a).compute()
# train_x['LotShape'] = le.fit_transform(train_x['LotShape'])

In [13]:
# train_x.head()

In [14]:
def keyval(allkeys):
    i = 0
    keyvalpairs = list()
    for key in allkeys:
        keyvalpairs.append((key,i))
        i+=1
    return dict(keyvalpairs)

In [15]:
# cols = ['FireplaceQu', 'LotShape']
cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
keys = {}
for col in cols:
    key = train_x[col].unique()
    keyvalpairs = keyval(key)
    keys.update({col:keyvalpairs})


In [16]:
keys

{'FireplaceQu': {0: 0, 'TA': 1, 'Gd': 2, 'Fa': 3, 'Ex': 4, 'Po': 5},
 'BsmtQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 0: 3, 'Fa': 4},
 'BsmtCond': {'TA': 0, 'Gd': 1, 0: 2, 'Fa': 3, 'Po': 4},
 'GarageQual': {'TA': 0, 'Fa': 1, 'Gd': 2, 0: 3, 'Ex': 4, 'Po': 5},
 'GarageCond': {'TA': 0, 'Fa': 1, 0: 2, 'Gd': 3, 'Po': 4, 'Ex': 5},
 'ExterQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'ExterCond': {'TA': 0, 'Gd': 1, 'Fa': 2, 'Po': 3, 'Ex': 4},
 'HeatingQC': {'Ex': 0, 'Gd': 1, 'TA': 2, 'Fa': 3, 'Po': 4},
 'PoolQC': {0: 0, 'Ex': 1, 'Fa': 2, 'Gd': 3},
 'KitchenQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'BsmtFinType1': {'GLQ': 0,
  'ALQ': 1,
  'Unf': 2,
  'Rec': 3,
  'BLQ': 4,
  0: 5,
  'LwQ': 6},
 'BsmtFinType2': {'Unf': 0,
  'BLQ': 1,
  0: 2,
  'ALQ': 3,
  'Rec': 4,
  'LwQ': 5,
  'GLQ': 6},
 'Functional': {'Typ': 0,
  'Min1': 1,
  'Maj1': 2,
  'Min2': 3,
  'Mod': 4,
  'Maj2': 5,
  'Sev': 6},
 'Fence': {0: 0, 'MnPrv': 1, 'GdWo': 2, 'GdPrv': 3, 'MnWw': 4},
 'BsmtExposure': {'No': 0, 'Gd': 1, 'Mn': 2, '

In [17]:
train_X = train_x.replace(keys).copy()

In [18]:
train_X['FireplaceQu'].unique().compute()

0    0
1    1
2    2
3    3
4    4
5    5
Name: FireplaceQu, dtype: int64

In [19]:
train_X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,0,RL,65.0,8450.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,0,WD,Normal
1,2.0,1,RL,80.0,9600.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,1,1,WD,Normal
2,3.0,0,RL,68.0,11250.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2,0,WD,Normal
3,4.0,2,RL,60.0,9550.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,2,WD,Abnorml
4,5.0,0,RL,84.0,14260.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,3,0,WD,Normal


In [20]:
cols_to_remove = []
for col in train_X.columns:
#     print(type(col))
    try:
        a = train_X[col].astype(float).compute()
    except ValueError:
#         print('Couldn\'t covert %s to float' % col)
        cols_to_remove.append(col)
        pass

# keep only the columns in df that do not contain string
train_X = train_X[[col for col in train_X.columns if col not in cols_to_remove]]

In [21]:
print(cols_to_remove)

['MSZoning', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']


In [22]:
train_X = train_X.drop(["Id"], axis = 1)

In [23]:
# tsklearn.preprocessing.MinMaxScalerrain_X.head()

In [24]:
len(train_X.columns)

58

In [25]:
len(dataset_list[1].columns)

1

In [26]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_X)
train_X_scaled = scaler_x.transform(train_X)

scaler_y = MinMaxScaler()
scaler_y.fit(dataset_list[1])
train_Y_scaled = scaler_y.transform(dataset_list[1])

In [27]:
# col = train_X.columns.to_list()
# scaler_d = MinMaxScaler()
# train_X.columns = list(train_X.columns)
# scaled_train_X = scaler_d.fit_transform(train_X[col])

In [28]:
train_X_scaled
# train_Y_scaled

array([[0.        , 0.20766773, 0.0334198 , ..., 0.        , 0.        ,
        0.        ],
       [0.07142857, 0.25559105, 0.03879502, ..., 0.        , 0.09090909,
        0.25      ],
       [0.        , 0.2172524 , 0.04650728, ..., 0.        , 0.18181818,
        0.        ],
       ...,
       [0.14285714, 0.21086262, 0.03618687, ..., 0.16129032, 0.09090909,
        1.        ],
       [0.07142857, 0.2172524 , 0.03934189, ..., 0.        , 0.63636364,
        1.        ],
       [0.07142857, 0.23961661, 0.04037019, ..., 0.        , 1.        ,
        0.        ]])

In [29]:
train_Y_scaled

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [30]:
print(train_X_scaled.shape)
print(train_Y_scaled.shape)

(1460, 58)
(1460, 1)


In [31]:
import numpy as np
total_data = np.append(train_X_scaled, train_Y_scaled, axis = 1)

In [32]:
print(total_data.shape)

(1460, 59)


In [33]:
from sklearn.model_selection import train_test_split
(train, test) = train_test_split(total_data, test_size=0.25, random_state=42)

In [34]:
trainX = train[:,:58]
trainY = train[:,-1]
testX = test[:,:58]
testY = test[:,-1]
print(trainX.shape)
print(trainY.shape)

(1095, 58)
(1095,)


In [35]:
train_Y_scaled

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [36]:
trainY = trainY.reshape((trainY.shape[0],1))
print(trainY.shape)

(1095, 1)


In [37]:
testY = testY.reshape((testY.shape[0],1))

In [38]:
start = time.time()
m = multiple.Multiple_Model_Gen_V3(trainX, trainY, testX, testY, 10, 128, input_shape = 58, 
                                   max_no_layers = 3, model_per_batch = 10, 
                                   save_dir = "/home/anish/ASC_ML_test_weights/")
m.get_model_confs()
m.get_best_models()
end = time.time()
print("Time Taken : ", (end-start))

2022-08-18 16:19:54.241174: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-18 16:19:54.241896: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-18 16:19:54.295734: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-18 16:19:54.295873: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.695GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-08-18 16:19:54.295888: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-08-18 16:19:54.297186: I tensorflow/stream_executor/platform/de


 
 

loss  :  0.5614935159683228 , TEST :  0.6082839965820312
output_layer_dense_16_16_loss  :  0.06931417435407639 , TEST :  0.07220642268657684
output_layer_dense_16_32_loss  :  0.052942048758268356 , TEST :  0.05933201313018799
output_layer_dense_16_64_loss  :  0.060586415231227875 , TEST :  0.0681532770395279
output_layer_dense_16_128_loss  :  0.056907862424850464 , TEST :  0.0638466328382492
output_layer_dense_16_196_loss  :  0.04519735276699066 , TEST :  0.04804239794611931
output_layer_dense_16_256_loss  :  0.060010481625795364 , TEST :  0.06379079073667526
output_layer_dense_32_16_loss  :  0.0620046965777874 , TEST :  0.06528446078300476
output_layer_dense_32_32_loss  :  0.05562868341803551 , TEST :  0.06005492061376572
output_layer_dense_32_64_loss  :  0.05505463853478432 , TEST :  0.05754172429442406
output_layer_dense_32_128_loss  :  0.043847158551216125 , TEST :  0.05003136396408081

 
 

loss  :  0.420859694480896 , TEST :  0.507727324962616
output_layer_dense_32_196_loss


 
 

loss  :  0.4231360852718353 , TEST :  0.5153645277023315
output_layer_dense_32_196_256_loss  :  0.042002320289611816 , TEST :  0.05282604321837425
output_layer_dense_32_256_16_loss  :  0.04428566247224808 , TEST :  0.05139881372451782
output_layer_dense_32_256_32_loss  :  0.04656387120485306 , TEST :  0.05697561800479889
output_layer_dense_32_256_64_loss  :  0.041405126452445984 , TEST :  0.052937038242816925
output_layer_dense_32_256_128_loss  :  0.03905055299401283 , TEST :  0.05082831159234047
output_layer_dense_32_256_196_loss  :  0.03919053450226784 , TEST :  0.048840828239917755
output_layer_dense_32_256_256_loss  :  0.03861147165298462 , TEST :  0.04808247089385986
output_layer_dense_64_16_16_loss  :  0.048133160918951035 , TEST :  0.05143145099282265
output_layer_dense_64_32_16_loss  :  0.040488239377737045 , TEST :  0.04923305660486221
output_layer_dense_64_32_32_loss  :  0.043405164033174515 , TEST :  0.05281093716621399

 
 

loss  :  0.38747718930244446 , TEST :  0.48


 
 

loss  :  0.31110066175460815 , TEST :  0.4524959325790405
output_layer_dense_256_196_16_loss  :  0.032393816858530045 , TEST :  0.04557625576853752
output_layer_dense_256_196_32_loss  :  0.030033063143491745 , TEST :  0.044259440153837204
output_layer_dense_256_196_64_loss  :  0.030451850965619087 , TEST :  0.04397163540124893
output_layer_dense_256_196_128_loss  :  0.03345393016934395 , TEST :  0.04487281292676926
output_layer_dense_256_196_196_loss  :  0.024548495188355446 , TEST :  0.04240965470671654
output_layer_dense_256_256_16_loss  :  0.032515231519937515 , TEST :  0.044820696115493774
output_layer_dense_256_256_32_loss  :  0.03384191542863846 , TEST :  0.046558912843465805
output_layer_dense_256_256_64_loss  :  0.02994370087981224 , TEST :  0.04537368565797806
output_layer_dense_256_256_128_loss  :  0.03250366449356079 , TEST :  0.048815205693244934
output_layer_dense_256_256_196_loss  :  0.03141497075557709 , TEST :  0.045837581157684326

 
 

loss  :  0.023594988510012

In [39]:
a = (int)t

SyntaxError: invalid syntax (591228447.py, line 1)

In [ ]:
from tensorflow.keras.activations import tanh, relu

## 2,3 Hidden Layers, 100 epochs, time taken = 147s (model saved)

In [ ]:
start = time.time()
m = multiple.Multiple_Model_Gen_V3(trainX, trainY, testX, testY, 10, 128, input_shape = 58, 
                                   max_no_layers = 3, model_per_batch = 10, 
                                   save_dir = "/home/anish/ASC_ML_test_weights/")
m.get_model_confs()
m.get_best_models()
end = time.time()
print("Time Taken : ", (end-start))

In [40]:
m.evaluate_dict_list

[{'model_name': 'dense_256_128_128',
  'score': 0.02388465590775013,
  'path_weights': '/home/anish/ASC_ML_test_weights/dense_256_128_128',
  'model': <tensorflow.python.keras.engine.functional.Functional at 0x7f5a50046280>},
 {'model_name': 'dense_256_196_196',
  'score': 0.024548495188355446,
  'path_weights': '/home/anish/ASC_ML_test_weights/dense_256_196_196',
  'model': <tensorflow.python.keras.engine.functional.Functional at 0x7f5a58453be0>},
 {'model_name': 'dense_196_256_128',
  'score': 0.0247303768992424,
  'path_weights': '/home/anish/ASC_ML_test_weights/dense_196_256_128',
  'model': <tensorflow.python.keras.engine.functional.Functional at 0x7f5a5846d5e0>},
 {'model_name': 'dense_196_196_128',
  'score': 0.025010880082845688,
  'path_weights': '/home/anish/ASC_ML_test_weights/dense_196_196_128',
  'model': <tensorflow.python.keras.engine.functional.Functional at 0x7f59edf2cca0>},
 {'model_name': 'dense_196_196_32',
  'score': 0.025457432493567467,
  'path_weights': '/home/a